In [13]:
import psycopg2
import numpy as np
from os import environ, chdir; chdir('/home/jovyan/work')
conn = psycopg2.connect(dbname='postgres', user='postgres', host=environ['REDHAT_POSTGRES_1_PORT_5432_TCP_ADDR'])
cur = conn.cursor()

In [14]:
cur.execute("SELECT COUNT(*) FROM one_hot_ppl_act"); print(cur.fetchone())

(7287,)


In [15]:
from lib.helpers.database_helper import keys, one_hot_from_table

In [16]:
keys = ', '.join([key[0] for key in keys])
cur.execute("SELECT ({}) FROM one_hot_ppl_act WHERE (act_outcome = true OR act_outcome = false)".format(keys))

ProgrammingError: column "act_category" does not exist
LINE 1: SELECT (act_id, act_date, act_category, act_char_1, act_char...
                                  ^


In [7]:
one_hot_from_table(cur)

TypeError: 'NoneType' object is not subscriptable

In [32]:
this_one_hot_vec, this_outcome = one_hot_from_table(cur)

In [8]:
import os; os.chdir('../')
from numpy.random import random
from lib.nn.perceptron import Perceptron

In [33]:
p = Perceptron(random(len(this_one_hot_vec)+1))

In [34]:
p.activate(this_one_hot_vec)

array(1.0)

In [35]:
this_outcome

0.0

In [65]:
def one_hot_from_table(cursor):
    one_hot_vec = (cursor.fetchone()[0]
     .replace('t','1')
     .replace('f','0')
     .replace('(','')
     .replace(')','')
     .split(',')
    )
    one_hot_vec = [float(el) for el in one_hot_vec]
    outcome = one_hot_vec[-1:][0]
    one_hot_vec = one_hot_vec[:-1]
    return np.array(one_hot_vec), outcome

this_one_hot_vec, this_outcome = one_hot_from_table(cur)

p = Perceptron(random(len(this_one_hot_vec)+1))

In [66]:
%%timeit


cur.execute("SELECT ({}) FROM one_hot_ppl_act WHERE (act_outcome = true OR act_outcome = false)".format(keys))

for _ in range(7000):
    this_one_hot_vec, this_outcome = one_hot_from_table(cur)
    p.update(this_one_hot_vec,this_outcome)

1 loop, best of 3: 2.08 s per loop


In [67]:
this_one_hot_vec, this_outcome = one_hot_from_table(cur)
p.activate(this_one_hot_vec), this_outcome

(array(0.0), 0.0)